## DataFrame

处理系列的许多方面延续到DataFrame。pandas的DataFrame让我们轻松地管理我们的数据和他们的直觉结构。 

DataFrames可以容纳多种类型的数据,但DataFrames是二维的对象,不同于series。

### 转换DataFrame：
字典转DataFrames的例子：

In [ ]:
import numpy as np
import pandas as pd

dict_data = {
'a' : [1, 2, 3, 4, 5],
'b' : ['L', 'K', 'J', 'M', 'Z'],
'c' : np.random.normal(0, 1, 5)
}
print (dict_data)
frame_data = pd.DataFrame(dict_data, index=pd.date_range('20160101',periods=5))
print (frame_data)

Series组合成DataFrame的例子：

In [ ]:
s_1 = pd.Series([2, 4, 6, 8, 10], name='APPL')
s_2 = pd.Series([1, 3, 5, 7, 9], name="GOOG")
numbers = pd.concat([s_1, s_2], axis=1)
print (numbers)
print (type(numbers))

### 修改dataframes的列名、索引；读取dataframes的值：

In [ ]:
print (numbers.columns)
# 修改列名
numbers.columns = ['MSFT', 'YHOO']
print (numbers)

In [ ]:
print (numbers.index)
# 修改索引
numbers.index = pd.date_range("20160101",periods=len(numbers))
print (numbers)
# 读取Dataframe的值
print (numbers.values)

### 访问DataFrame元素

关键在于一切现在都要考虑多个维度。发生这种情况的主要途径是通过访问DataFrame单独或整组的列,。要做到这一点, 
我们可以通过直接访问属性或通过使用我们已经熟悉的方法。

用tushare请求A股数据，然后做成dataframe，最后通过loc与iloc读取需要的数据。

In [ ]:
stock1 = pd.read_excel('sz50.xlsx',sheetname='600036.XSHG', index_col='datetime')
stock2 = pd.read_excel('sz50.xlsx',sheetname='600050.XSHG', index_col='datetime')
stock3 = pd.read_excel('sz50.xlsx',sheetname='601318.XSHG', index_col='datetime')

In [ ]:
from datetime import datetime

symbol=['600036.XSHG','600050.XSHG','601318.XSHG']
data_dict = {}
for s in symbol:
    data =  pd.read_excel('sz50.xlsx',sheetname=s, index_col='datetime')
    data_dict[s] = data['close']

In [ ]:
data = pd.DataFrame(data_dict)
print(data.loc[datetime(2017,1,1):datetime(2017,1,10),['600036.XSHG', '601318.XSHG']])
print(data.iloc[0:2,1])
print(data.iloc[[1, 3, 5] + list(range(7, 20, 2)), [0, 1]].head(20))

### 布尔索引

与Series一样,有时候我们想过滤DataFrame根据一组标准。我们通过索引DataFrame布尔值。

In [ ]:
print(data.loc[data['600036.XSHG'].pct_change() > data['601318.XSHG'].pct_change()].head())

### 添加、删除列,结合DataFrames /Series

当你已经有一个DataFrame的数据,这很好,但同样重要的是能够增加你的数据。 
添加新数据：

In [ ]:
new = pd.read_excel('sz50.xlsx',sheetname='600519.XSHG', index_col='datetime')
data['600519.XSHG'] = new.close
print(data.head(5))

删除某一行

In [ ]:
data = data.drop('600050.XSHG', axis=1)
print(data.head(5))

合并某一行

In [ ]:
gold_stock = pd.read_excel('sz50.xlsx',sheetname='600547.XSHG', index_col='datetime')

df=pd.concat([data,gold_stock['close']], axis=1)
print(df.head(5))

In [ ]:
df.rename(columns={'close':'600547.XSHG'}, inplace = True)

In [ ]:
print(df.head(5))

### 缺失的数据(再一次)

把一个真实数据输入DataFrame带给我们与在系列中同样的问题,只是这次更多的维度。我们有和系列相同的方法来访问,如下显示。

In [ ]:
print(df[df.isnull().values==True])

In [ ]:
df_na = df.fillna(method='ffill')

In [ ]:
print(df_na.loc['2017-04-26':'2017-05-17'])

### 时间序列分析

使用安装在内部的统计方法来计算DataFrames,我们可以对多个时间序列进行计算。执行计算的代码在DataFrames与在series上几乎一模一样,所以不要担心再学习一切。

将DataFrame数据可视化：

In [ ]:
import matplotlib.pyplot as plt

onebegin=data/data.iloc[0]

plt.figure(figsize=(15, 7))
plt.plot(onebegin)
plt.title("Onebegin Stock Prices")
plt.ylabel("Price")
plt.xlabel("Date")
plt.show()

对dataframe数据进行计算：

In [ ]:
print('mean:','\n',data.mean(axis=0))
print('std:','\n',data.std(axis=0))
print(onebegin.head(5))

将回报率标准化，然后可视化。

In [ ]:
mult_returns = data.pct_change()[1:]
print(mult_returns.head(5))

In [ ]:
norm_returns = (mult_returns-mult_returns.mean(axis=0))/mult_returns.std(axis=0)

plt.figure(figsize=(15, 7))
plt.plot(norm_returns)
plt.hlines(0, norm_returns.index[0],norm_returns.index[-1], linestyles='dashed')
plt.show()

将dataframe里的数据计算40均线，最后可视化展示出来：

In [ ]:
rolling_mean = data['601318.XSHG'].rolling(window=40,center=False).mean()

plt.figure(figsize=(15, 7))
data['601318.XSHG'].plot()
rolling_mean.plot()
plt.title("40days Rolling Mean of 601318.XSHG")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()
plt.show()